<a href="https://colab.research.google.com/github/wose70/HDB_Rental/blob/main/Python_Script_Rental_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Python Implementation Guide for Rental Price Prediction System

# This guide provides a comprehensive implementation for a rental price prediction system using
# the provided Singapore rental market datasets.

# 1. Setup and Import Libraries
# -----------------------------
# Import necessary libraries for data manipulation (pandas, numpy), visualization (matplotlib, seaborn, folium),
# machine learning (scikit-learn), deep learning (tensorflow.keras), and geospatial analysis (geopandas).
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium # For interactive maps
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV # For splitting data, cross-validation, hyperparameter tuning
from sklearn.preprocessing import StandardScaler, OneHotEncoder # For scaling numerical data and encoding categorical data
from sklearn.compose import ColumnTransformer # To apply different transformations to different columns
from sklearn.pipeline import Pipeline # To chain preprocessing and model steps
from sklearn.linear_model import LinearRegression, Ridge, Lasso # Linear models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor # Ensemble models
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error # Evaluation metrics
import xgboost as xgb # XGBoost library (often needs separate installation: pip install xgboost)
# Note: The script uses 'XGBRegressor' alias later, let's keep it consistent
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential # For building neural networks layer by layer
from tensorflow.keras.layers import Dense, Dropout # Core layers for a neural network
from tensorflow.keras.optimizers import Adam # Optimizer for training neural networks
import geopandas as gpd # For working with geospatial data
from shapely.geometry import Point # For creating geometric points from coordinates
import warnings
warnings.filterwarnings('ignore') # Suppress warnings (useful for cleaner output, but be cautious in production)

# 2. Data Loading
# ---------------
# Function to load the required datasets. Assumes specific file names.
# Make sure these files ('RentingOutofFlats2025.csv', 'street_coordinates.csv',
# 'monthly avg and median household income.xlsx') are in the same directory as the script,
# or provide the full path.
def load_data():
    """Loads the rental, street coordinates, and income datasets."""
    try:
        # Load the datasets
        rental_df = pd.read_csv('RentingOutofFlats2025.csv')
        street_coords_df = pd.read_csv('street_coordinates.csv')
        # Assuming the income data is in the first sheet of the Excel file
        income_df = pd.read_csv('/content/monthly avg and medium household income.csv')

        print(f"Rental data shape: {rental_df.shape}")
        print(f"Street coordinates data shape: {street_coords_df.shape}")
        print(f"Income data shape: {income_df.shape}")

        return rental_df, street_coords_df, income_df
    except FileNotFoundError as e:
        print(f"Error loading data: {e}. Make sure the data files are in the correct location.")
        return None, None, None

# 3. Data Exploration (EDA)
# -------------------------
# Function to perform initial Exploratory Data Analysis.
# Generates plots to understand data distributions and relationships.
def explore_data(rental_df, street_coords_df, income_df):
    """Performs basic EDA on the rental data and saves plots."""
    if rental_df is None:
        print("Rental data not loaded. Skipping exploration.")
        return None

    # Basic information about rental data
    print("\nRental Data Info:")
    rental_df.info() # Prints column names, non-null counts, and data types
    print("\nRental Data Sample:")
    print(rental_df.head()) # Shows the first 5 rows

    # Basic statistics for the target variable ('monthly_rent') grouped by 'flat_type'
    print("\nRental Price Statistics by Flat Type:")
    print(rental_df.groupby('flat_type')['monthly_rent'].describe())

    # Distribution of rental prices (Histogram)
    plt.figure(figsize=(10, 6))
    sns.histplot(rental_df['monthly_rent'], kde=True) # kde=True adds a density curve
    plt.title('Distribution of Monthly Rent Prices')
    plt.xlabel('Monthly Rent (SGD)')
    plt.ylabel('Frequency')
    plt.tight_layout() # Adjust layout to prevent labels overlapping
    plt.savefig('rent_distribution.png') # Save the plot as an image
    plt.close() # Close the plot to free up memory

    # Rental prices by town (Box Plot)
    plt.figure(figsize=(14, 8))
    # Sort towns by median rent for better visualization
    town_order = rental_df.groupby('town')['monthly_rent'].median().sort_values(ascending=False).index
    sns.boxplot(x='town', y='monthly_rent', data=rental_df, order=town_order)
    plt.title('Rental Prices by Town')
    plt.xlabel('Town')
    plt.ylabel('Monthly Rent (SGD)')
    plt.xticks(rotation=90) # Rotate x-axis labels for readability
    plt.tight_layout()
    plt.savefig('rent_by_town.png')
    plt.close()

    # Rental prices by flat type (Box Plot)
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='flat_type', y='monthly_rent', data=rental_df)
    plt.title('Rental Prices by Flat Type')
    plt.xlabel('Flat Type')
    plt.ylabel('Monthly Rent (SGD)')
    plt.tight_layout()
    plt.savefig('rent_by_flat_type.png')
    plt.close()

    # Check for temporal trends
    # Convert 'rent_approval_date' to datetime objects, inferring format if possible
    try:
        rental_df['rent_approval_date'] = pd.to_datetime(rental_df['rent_approval_date'])
        rental_df['year_month'] = rental_df['rent_approval_date'].dt.to_period('M') # Extract Year-Month

        # Calculate average rent per month for each flat type
        time_trend = rental_df.groupby(['year_month', 'flat_type'])['monthly_rent'].mean().unstack()

        plt.figure(figsize=(12, 8))
        time_trend.plot(marker='o', ax=plt.gca()) # Plot on the current axes
        plt.title('Average Monthly Rent Over Time by Flat Type')
        plt.xlabel('Year-Month')
        plt.ylabel('Average Monthly Rent (SGD)')
        plt.legend(title='Flat Type')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig('rent_time_trend.png')
        plt.close()
    except KeyError:
        print("Column 'rent_approval_date' not found. Skipping time trend analysis.")
    except Exception as e:
        print(f"Error processing date for time trend: {e}")


    return rental_df # Return the potentially modified dataframe (with new date columns)

# 4. Data Integration
# -------------------
# Function to merge the different data sources together.
def integrate_data(rental_df, street_coords_df, income_df):
    """Merges rental, coordinates, and income data."""
    if rental_df is None or street_coords_df is None:
        print("Required dataframes for integration not loaded. Skipping.")
        return None

    # Merge rental data with street coordinates based on 'street_name'
    # Using 'left' merge keeps all rows from rental_df
    merged_df = pd.merge(
        rental_df,
        street_coords_df,
        on='street_name', # Assumes 'street_name' is the common column
        how='left'
    )

    print(f"\nShape after merging with coordinates: {merged_df.shape}")
    print(f"Missing values after merging with coordinates:\n{merged_df.isnull().sum()}")

    # Handle missing coordinates (latitude, longitude) after the merge
    # Strategy: Fill missing coordinates for a street with the average for that street.
    # If still missing (e.g., street not in coords_df), fill with the average for the town.
    # If still missing, fill with the overall average coordinate.
    for col in ['latitude', 'longitude']:
        if merged_df[col].isnull().sum() > 0:
            # Fill with street average (use transform to broadcast the mean back to the original shape)
            street_coords_mean = merged_df.groupby('street_name')[col].transform('mean')
            merged_df[col].fillna(street_coords_mean, inplace=True)

            # Fill remaining NAs with town average
            town_coords_mean = merged_df.groupby('town')[col].transform('mean')
            merged_df[col].fillna(town_coords_mean, inplace=True)

            # Fill any remaining NAs with the overall mean
            overall_mean = merged_df[col].mean()
            merged_df[col].fillna(overall_mean, inplace=True)
            print(f"Missing values for {col} after imputation: {merged_df[col].isnull().sum()}")


    # Integrate Income Data (Placeholder - Needs Adjustment based on actual income_df structure)
    # This section assumes 'income_df' has 'town' and 'median_household_income' columns.
    # You MUST inspect 'income_df' and adjust the merging logic accordingly.
    if income_df is not None and 'town' in income_df.columns and 'median_household_income' in income_df.columns:
        # Example: Aggregate income data by town (e.g., taking the mean if multiple entries per town)
        # Adjust the aggregation method (mean, median, first, etc.) as needed.
        income_summary = income_df.groupby('town')['median_household_income'].mean().reset_index()

        # Merge with the main dataframe
        merged_df = pd.merge(
            merged_df,
            income_summary,
            on='town',
            how='left' # Keep all rental records, add income where available
        )

        # Handle missing income data after merge (e.g., fill with overall median/mean)
        if 'median_household_income' in merged_df.columns and merged_df['median_household_income'].isnull().sum() > 0:
            median_income = merged_df['median_household_income'].median()
            merged_df['median_household_income'].fillna(median_income, inplace=True)
            print(f"Missing values for median_household_income after imputation: {merged_df['median_household_income'].isnull().sum()}")
        print(f"\nShape after merging with income data: {merged_df.shape}")

    else:
        print("\nIncome data not loaded or required columns ('town', 'median_household_income') not found. Skipping income integration.")


    return merged_df

# 5. Feature Engineering
# ----------------------
# Function to create new features from existing ones.
def engineer_features(merged_df):
    """Creates new features from the merged data."""
    if merged_df is None:
        print("Merged dataframe not available. Skipping feature engineering.")
        return None

    # --- Date Features ---
    # Extract year and month from 'rent_approval_date' if it exists and is datetime
    if 'rent_approval_date' in merged_df.columns and pd.api.types.is_datetime64_any_dtype(merged_df['rent_approval_date']):
        merged_df['approval_year'] = merged_df['rent_approval_date'].dt.year
        merged_df['approval_month'] = merged_df['rent_approval_date'].dt.month
        # Optionally drop the original date column if no longer needed for direct modeling
        # merged_df = merged_df.drop('rent_approval_date', axis=1)
    else:
         print("Column 'rent_approval_date' not found or not datetime. Skipping date feature extraction.")


    # --- Geospatial Features ---
    # Calculate distance to a central point (e.g., CBD)
    # Using approximate coordinates for Singapore CBD (Raffles Place)
    cbd_lat, cbd_lon = 1.2830, 103.8513 # More precise Raffles Place MRT approx. coords

    # Check if latitude and longitude columns exist
    if 'latitude' in merged_df.columns and 'longitude' in merged_df.columns:
         # Simple Euclidean distance approximation (works reasonably well for small areas like Singapore)
         # For higher accuracy over larger distances, use Haversine formula
         merged_df['dist_to_cbd_approx'] = np.sqrt(
             (merged_df['latitude'] - cbd_lat)**2 +
             (merged_df['longitude'] - cbd_lon)**2
         )
         # Convert approx distance to km (1 degree latitude ~ 111km, 1 degree longitude ~ 111km * cos(latitude))
         # Using a simplified factor for Singapore's latitude (~1.35 deg N)
         approx_km_factor = 111 * np.cos(np.radians(1.35))
         merged_df['dist_to_cbd_km'] = merged_df['dist_to_cbd_approx'] * approx_km_factor

    else:
        print("Latitude or Longitude columns missing. Skipping distance to CBD calculation.")


    # --- Flat Type Features ---
    # Extract number of rooms from 'flat_type' (e.g., '3-ROOM' -> 3)
    # Handles cases like 'MULTI-GENERATION' or 'EXECUTIVE' where there's no leading digit
    if 'flat_type' in merged_df.columns:
        merged_df['room_count'] = merged_df['flat_type'].str.extract('^(\d+)').astype(float)
        # Handle non-standard flat types (e.g., EXECUTIVE, MULTI-GENERATION) - fill with a reasonable value or median/mode
        # Example: Fill NaN room_count with the median room count
        median_rooms = merged_df['room_count'].median()
        merged_df['room_count'].fillna(median_rooms, inplace=True)
    else:
        print("Column 'flat_type' not found. Skipping room count extraction.")


    # --- Regional Features ---
    # Create broader geographical regions based on towns
    # This mapping should be verified or refined based on official planning areas if possible.
    if 'town' in merged_df.columns:
        region_mapping = {
            'ANG MO KIO': 'North-East', 'BEDOK': 'East', 'BISHAN': 'Central', 'BUKIT BATOK': 'West',
            'BUKIT MERAH': 'Central', 'BUKIT PANJANG': 'West', 'BUKIT TIMAH': 'Central', # Note: Bukit Timah often considered Central
            'CENTRAL AREA': 'Central', 'CHOA CHU KANG': 'West', 'CLEMENTI': 'West', 'GEYLANG': 'Central', # Note: Geylang often considered Central fringe
            'HOUGANG': 'North-East', 'JURONG EAST': 'West', 'JURONG WEST': 'West', 'KALLANG/WHAMPOA': 'Central',
            'MARINE PARADE': 'East', 'PASIR RIS': 'East', 'PUNGGOL': 'North-East', 'QUEENSTOWN': 'Central',
            'SEMBAWANG': 'North', 'SENGKANG': 'North-East', 'SERANGOON': 'North-East', 'TAMPINES': 'East',
            'TOA PAYOH': 'Central', 'WOODLANDS': 'North', 'YISHUN': 'North'
        }
        merged_df['region'] = merged_df['town'].map(region_mapping)
        # Handle any towns not in the mapping (assign to 'Other' or a default region)
        merged_df['region'].fillna('Other', inplace=True)
    else:
        print("Column 'town' not found. Skipping region mapping.")

    # --- Clean up / Drop unnecessary columns ---
    # Drop columns that were used for intermediate steps or are redundant
    cols_to_drop = ['rent_approval_date', 'year_month', 'street_name', 'block', 'dist_to_cbd_approx'] # Example list
    # Drop only if they exist
    merged_df = merged_df.drop(columns=[col for col in cols_to_drop if col in merged_df.columns], axis=1)

    print("\nFeatures after engineering:")
    print(merged_df.head())
    print(f"\nShape after feature engineering: {merged_df.shape}")

    return merged_df


# 6. Model Building (Traditional ML)
# ----------------------------------
# Function to preprocess data, train, and evaluate standard ML models.
def build_models(processed_df):
    """Builds, trains, and evaluates traditional ML regression models."""
    if processed_df is None:
        print("Processed dataframe not available. Skipping model building.")
        return {}, None, None, None, None

    try:
        # Define features (X) and target (y)
        X = processed_df.drop('monthly_rent', axis=1)
        y = processed_df['monthly_rent']

        # Identify categorical and numerical columns *after* feature engineering
        categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
        # Ensure boolean columns are treated as numerical or converted appropriately if needed
        numerical_cols = X.select_dtypes(include=np.number).columns.tolist() # Includes int, float, potentially bool

        print(f"\nNumerical features: {numerical_cols}")
        print(f"Categorical features: {categorical_cols}")

        # Create a preprocessing pipeline
        # Numerical features: Impute missing values (e.g., with median) and scale (e.g., StandardScaler)
        # Categorical features: Impute missing values (e.g., with 'missing' category) and one-hot encode
        # Note: Imputation might have been done earlier, but adding here makes the pipeline robust
        from sklearn.impute import SimpleImputer

        numerical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')), # Impute missing numerical values
            ('scaler', StandardScaler()) # Scale numerical values
        ])

        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), # Impute missing categorical values
            ('onehot', OneHotEncoder(handle_unknown='ignore')) # One-hot encode, ignore categories not seen in training
        ])

        # Combine preprocessing steps using ColumnTransformer
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_cols),
                ('cat', categorical_transformer, categorical_cols)
            ],
            remainder='passthrough' # Keep other columns (if any) - should ideally be none if cols are defined correctly
            )

        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        print(f"\nTraining set shape: {X_train.shape}")
        print(f"Testing set shape: {X_test.shape}")

        # Define models to evaluate
        models = {
            'Linear Regression': Pipeline([
                ('preprocessor', preprocessor),
                ('regressor', LinearRegression())
            ]),
            'Ridge': Pipeline([
                ('preprocessor', preprocessor),
                # Ridge adds L2 regularization - alpha controls strength
                ('regressor', Ridge(alpha=1.0, random_state=42))
            ]),
            'Random Forest': Pipeline([
                ('preprocessor', preprocessor),
                # Ensemble model - n_estimators = number of trees
                # n_jobs=-1 uses all available CPU cores
                ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, max_depth=20, min_samples_split=10)) # Added some common hyperparameters
            ]),
            'XGBoost': Pipeline([
                ('preprocessor', preprocessor),
                # Gradient Boosting model - often high performance
                ('regressor', XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, n_jobs=-1, max_depth=7, subsample=0.8)) # Added some common hyperparameters
            ])
        }

        # Train and evaluate each model
        results = {}
        for name, model in models.items():
            print(f"\n--- Training {name} ---")
            model.fit(X_train, y_train)

            # Make predictions on the test set
            y_pred = model.predict(X_test)

            # Evaluate using regression metrics
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse) # Root Mean Squared Error
            mae = mean_absolute_error(y_test, y_pred) # Mean Absolute Error
            r2 = r2_score(y_test, y_pred) # R-squared (coefficient of determination)

            results[name] = {
                'model': model, # Store the trained pipeline
                'mse': mse,
                'rmse': rmse,
                'mae': mae,
                'r2': r2
            }

            print(f"{name} Results:")
            print(f"  RMSE: {rmse:.2f}") # Lower is better
            print(f"  MAE: {mae:.2f}")  # Lower is better (interpretable in original units)
            print(f"  R²: {r2:.4f}")   # Closer to 1 is better

        # Find best model based on RMSE (you could choose MAE or R2 as well)
        best_model_name = min(results, key=lambda k: results[k]['rmse'])
        print(f"\n== Best Traditional Model (based on RMSE): {best_model_name} with RMSE = {results[best_model_name]['rmse']:.2f} ==")

        return results, X_train, X_test, y_train, y_test, preprocessor # Return preprocessor too

    except Exception as e:
        print(f"An error occurred during model building: {e}")
        import traceback
        traceback.print_exc()
        return {}, None, None, None, None, None


# 7. Neural Network Model (Deep Learning)
# ---------------------------------------
# Function to build, train, and evaluate a simple feed-forward neural network.
def build_neural_network(X_train, y_train, X_test, y_test, preprocessor):
    """Builds, trains, and evaluates a neural network regression model."""
    if X_train is None or preprocessor is None:
         print("Training data or preprocessor not available. Skipping neural network.")
         return None, None

    try:
        print("\n--- Training Neural Network ---")
        # Apply the *already fitted* preprocessor to the training and test data
        # Important: Use transform, not fit_transform on test data!
        X_train_processed = preprocessor.transform(X_train)
        X_test_processed = preprocessor.transform(X_test)

        # Convert sparse matrix output from OneHotEncoder to dense array if necessary
        if hasattr(X_train_processed, "toarray"):
            X_train_processed = X_train_processed.toarray()
        if hasattr(X_test_processed, "toarray"):
            X_test_processed = X_test_processed.toarray()

        # Define the neural network architecture
        input_dim = X_train_processed.shape[1] # Number of features after preprocessing

        model = Sequential([
            Dense(128, activation='relu', input_shape=(input_dim,)), # Input layer + first hidden layer
            Dropout(0.2), # Dropout for regularization (prevents overfitting)
            Dense(64, activation='relu'), # Second hidden layer
            Dropout(0.2),
            Dense(32, activation='relu'), # Third hidden layer
            Dense(1) # Output layer (1 neuron for regression, linear activation by default)
        ])

        # Compile the model
        # Adam optimizer is common and effective. Loss 'mse' is standard for regression.
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae']) # mean_absolute_error

        model.summary() # Print model architecture

        # Set up Early Stopping to prevent overfitting
        # Monitors validation loss and stops if it doesn't improve for 'patience' epochs.
        # restore_best_weights=True ensures the model weights from the best epoch are kept.
        from tensorflow.keras.callbacks import EarlyStopping
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

        # Train the model
        print("Training NN...")
        history = model.fit(
            X_train_processed, y_train,
            epochs=100, # Maximum number of epochs
            batch_size=32, # Number of samples per gradient update
            validation_split=0.2, # Use 20% of training data for validation during training
            callbacks=[early_stop],
            verbose=2 # Show one line per epoch
        )

        # Evaluate the trained model on the test set
        loss, mae = model.evaluate(X_test_processed, y_test, verbose=0)
        rmse = np.sqrt(loss) # Since loss is MSE
        y_pred_nn = model.predict(X_test_processed)
        r2_nn = r2_score(y_test, y_pred_nn)

        print("\nNeural Network Evaluation Results:")
        print(f"  RMSE: {rmse:.2f}")
        print(f"  MAE: {mae:.2f}")
        print(f"  R²: {r2_nn:.4f}")

        # Plot training history (Loss and MAE over epochs)
        plt.figure(figsize=(12, 5))

        # Plot Loss
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss During Training')
        plt.ylabel('Mean Squared Error (Loss)')
        plt.xlabel('Epoch')
        plt.legend()

        # Plot MAE
        plt.subplot(1, 2, 2)
        plt.plot(history.history['mae'], label='Training MAE')
        plt.plot(history.history['val_mae'], label='Validation MAE')
        plt.title('Model MAE During Training')
        plt.ylabel('Mean Absolute Error')
        plt.xlabel('Epoch')
        plt.legend()

        plt.tight_layout()
        plt.savefig('nn_training_history.png')
        plt.close()

        nn_metrics = {'mse': loss, 'rmse': rmse, 'mae': mae, 'r2': r2_nn}

        # Note: The NN model itself doesn't include the preprocessing steps.
        # For prediction, you need both the preprocessor and the NN model.
        return model, nn_metrics

    except Exception as e:
        print(f"An error occurred during neural network training: {e}")
        import traceback
        traceback.print_exc()
        return None, None


# 8. Geospatial Analysis and Visualization
# ----------------------------------------
# Function to create maps and plots showing geographical patterns.
def create_geospatial_analysis(processed_df):
    """Creates geospatial visualizations like heatmaps and regional plots."""
    if processed_df is None or not all(col in processed_df.columns for col in ['latitude', 'longitude', 'monthly_rent', 'region', 'flat_type']):
        print("Required columns for geospatial analysis not found. Skipping.")
        return None

    print("\n--- Creating Geospatial Visualizations ---")

    # --- Heatmap of Rental Prices ---
    try:
        from folium.plugins import HeatMap

        # Create a base map centered on Singapore
        singapore_map_heat = folium.Map(location=[1.3521, 103.8198], zoom_start=11) # Slightly zoomed out

        # Create heat map data points: list of [latitude, longitude, intensity]
        # Filter out rows with missing coordinates
        heat_data = processed_df[['latitude', 'longitude', 'monthly_rent']].dropna().values.tolist()

        # Add heat map layer to the map
        HeatMap(heat_data, radius=10, blur=15).add_to(singapore_map_heat) # Adjust radius and blur as needed

        # Save the heatmap to an HTML file
        heatmap_filename = 'singapore_rent_heatmap.html'
        singapore_map_heat.save(heatmap_filename)
        print(f"Saved rental price heatmap to {heatmap_filename}")

    except ImportError:
        print("Folium or HeatMap plugin not found. Skipping heatmap generation. Install with: pip install folium")
    except Exception as e:
        print(f"Error creating heatmap: {e}")


    # --- Bar Plot: Average Rent by Region ---
    try:
        region_rent = processed_df.groupby('region')['monthly_rent'].mean().reset_index().sort_values('monthly_rent', ascending=False)

        plt.figure(figsize=(12, 7))
        sns.barplot(x='monthly_rent', y='region', data=region_rent, palette='viridis', orient='h')
        plt.title('Average Monthly Rental Prices by Region')
        plt.xlabel('Average Monthly Rent (SGD)')
        plt.ylabel('Region')
        plt.tight_layout()
        plt.savefig('avg_rent_by_region.png')
        plt.close()
        print("Saved average rent by region plot.")
    except Exception as e:
        print(f"Error creating rent by region plot: {e}")


    # --- Box Plot: Rent Distribution by Region and Flat Type ---
    # This can get crowded if there are many regions/types. Consider filtering or alternative plots.
    try:
        plt.figure(figsize=(16, 10))
        # Using town instead of region might be too granular, stick with region
        sns.boxplot(x='monthly_rent', y='region', hue='flat_type', data=processed_df, palette='tab10', orient='h')
        plt.title('Rental Price Distribution by Region and Flat Type')
        plt.xlabel('Monthly Rent (SGD)')
        plt.ylabel('Region')
        plt.legend(title='Flat Type', bbox_to_anchor=(1.05, 1), loc='upper left') # Place legend outside plot
        plt.tight_layout()
        plt.savefig('rent_dist_by_region_and_type.png')
        plt.close()
        print("Saved rent distribution by region and flat type plot.")
    except Exception as e:
        print(f"Error creating rent distribution plot: {e}")

    # Optional: Choropleth map requires a GeoJSON file defining region boundaries.
    # This is more complex and requires finding or creating a suitable GeoJSON for Singapore regions/towns.

# 9. Model Deployment Function (Conceptual)
# -----------------------------------------
# Function to wrap the prediction logic for new data.
def create_prediction_function(best_model, preprocessor):
    """Creates a function to predict rent for new input data."""
    if best_model is None or preprocessor is None:
        print("Model or preprocessor not available. Cannot create prediction function.")
        return None

    def predict_rent(input_features_dict):
        """
        Predicts rental price based on input features.

        Parameters:
        -----------
        input_features_dict : dict
            Dictionary containing feature names and values for a single prediction.
            Example: {'town': 'ANG MO KIO', 'flat_type': '4-ROOM', 'latitude': 1.37, 'longitude': 103.85, ...}
            Must include all features expected by the preprocessor.

        Returns:
        --------
        float or None
            Predicted monthly rent, or None if prediction fails.
        """
        try:
            # Convert input dictionary to pandas DataFrame (1 row)
            input_df = pd.DataFrame([input_features_dict])

            # --- Apply necessary feature engineering steps ---
            # This MUST mirror the steps in engineer_features applied to the training data
            # Example: Extract room count, map region, calculate distance to CBD
            # Ensure consistency! It's often better to wrap feature engineering in a reusable function/class.

            # Example: Re-apply some feature engineering (ensure consistency with training)
            if 'flat_type' in input_df.columns:
                 input_df['room_count'] = input_df['flat_type'].str.extract('^(\d+)').astype(float).fillna(input_df['room_count'].median()) # Use median from training if needed
            if 'town' in input_df.columns:
                # Use the same region mapping as in engineer_features
                region_mapping = {
                    'ANG MO KIO': 'North-East', 'BEDOK': 'East', 'BISHAN': 'Central', 'BUKIT BATOK': 'West',
                    'BUKIT MERAH': 'Central', 'BUKIT PANJANG': 'West', 'BUKIT TIMAH': 'Central',
                    'CENTRAL AREA': 'Central', 'CHOA CHU KANG': 'West', 'CLEMENTI': 'West', 'GEYLANG': 'Central',
                    'HOUGANG': 'North-East', 'JURONG EAST': 'West', 'JURONG WEST': 'West', 'KALLANG/WHAMPOA': 'Central',
                    'MARINE PARADE': 'East', 'PASIR RIS': 'East', 'PUNGGOL': 'North-East', 'QUEENSTOWN': 'Central',
                    'SEMBAWANG': 'North', 'SENGKANG': 'North-East', 'SERANGOON': 'North-East', 'TAMPINES': 'East',
                    'TOA PAYOH': 'Central', 'WOODLANDS': 'North', 'YISHUN': 'North'
                 }
                input_df['region'] = input_df['town'].map(region_mapping).fillna('Other')
            if 'latitude' in input_df.columns and 'longitude' in input_df.columns:
                cbd_lat, cbd_lon = 1.2830, 103.8513
                approx_km_factor = 111 * np.cos(np.radians(1.35))
                input_df['dist_to_cbd_approx'] = np.sqrt(((input_df['latitude'] - cbd_lat)**2) + ((input_df['longitude'] - cbd_lon)**2))
                input_df['dist_to_cbd_km'] = input_df['dist_to_cbd_approx'] * approx_km_factor

            # Ensure DataFrame has the same columns in the same order as the training data fed to the preprocessor
            # You might need to load the column order from training
            # X_train_cols = [...] # Load or define column list used for training X
            # input_df = input_df.reindex(columns=X_train_cols, fill_value=0) # Or appropriate fill value

            # --- Preprocess the input data using the *fitted* preprocessor ---
            input_processed = preprocessor.transform(input_df)

            # --- Make prediction ---
            # Handle potential differences if the best model is NN vs. traditional pipeline
            if isinstance(best_model, Pipeline): # If it's a scikit-learn pipeline
                 predicted_rent = best_model.predict(input_df)[0] # Pipeline handles preprocessing
            elif hasattr(best_model, 'predict') and not isinstance(best_model, Pipeline): # If it's a Keras model (or similar)
                 if hasattr(input_processed, "toarray"): # Convert sparse to dense if needed for NN
                     input_processed = input_processed.toarray()
                 predicted_rent = best_model.predict(input_processed)[0][0] # Keras predict might return nested array
            else:
                 print("Error: Unknown model type for prediction.")
                 return None

            return float(predicted_rent)

        except Exception as e:
            print(f"Error during prediction: {e}")
            import traceback
            traceback.print_exc()
            return None

    return predict_rent


# 10. Main Execution Block
# ------------------------
def main():
    """Main function to run the entire pipeline."""
    print("Starting Rental Price Prediction Pipeline...")

    # 1. Load Data
    print("\n[1/7] Loading data...")
    rental_df, street_coords_df, income_df = load_data()
    if rental_df is None: return # Stop if data loading failed

    # 2. Explore Data
    print("\n[2/7] Exploring data...")
    rental_df_explored = explore_data(rental_df.copy(), street_coords_df, income_df) # Use copy to avoid modifying original df in exploration
    if rental_df_explored is not None:
         rental_df = rental_df_explored # Keep date columns if added

    # 3. Integrate Data
    print("\n[3/7] Integrating data sources...")
    merged_df = integrate_data(rental_df, street_coords_df, income_df)
    if merged_df is None: return

    # 4. Engineer Features
    print("\n[4/7] Engineering features...")
    processed_df = engineer_features(merged_df.copy()) # Use copy before potential destructive operations
    if processed_df is None: return
    # Drop rows with NaN in target variable before modeling
    processed_df.dropna(subset=['monthly_rent'], inplace=True)


    # 5. Build Traditional Models
    print("\n[5/7] Building and evaluating traditional ML models...")
    model_results, X_train, X_test, y_train, y_test, preprocessor = build_models(processed_df)
    if not model_results: return # Stop if model building failed

    # 6. Build Neural Network Model
    print("\n[6/7] Building and evaluating neural network model...")
    # Pass the fitted preprocessor from the traditional models step
    nn_model, nn_metrics = build_neural_network(X_train, y_train, X_test, y_test, preprocessor)

    # --- Determine Overall Best Model ---
    best_model = None
    best_preprocessor_for_prediction = None # Need the preprocessor associated with the best model

    best_traditional_name = min(model_results, key=lambda k: model_results[k]['rmse'])
    best_traditional_rmse = model_results[best_traditional_name]['rmse']
    print(f"\nBest Traditional Model: {best_traditional_name} (RMSE: {best_traditional_rmse:.2f})")

    if nn_model and nn_metrics and nn_metrics['rmse'] < best_traditional_rmse:
        print(f"Neural Network is better (RMSE: {nn_metrics['rmse']:.2f})")
        best_model = nn_model # The Keras model object
        # The preprocessor used for NN was the same one fitted in build_models
        best_preprocessor_for_prediction = preprocessor
        best_model_type = 'Neural Network'
    else:
        print(f"{best_traditional_name} is the best overall model.")
        # The best model is the entire scikit-learn pipeline (preprocessor + regressor)
        best_model = model_results[best_traditional_name]['model']
        # For prediction using the pipeline, we don't need the preprocessor separately
        best_preprocessor_for_prediction = preprocessor # Still useful to have for potential separate use
        best_model_type = best_traditional_name

    # 7. Geospatial Analysis (using the final processed data)
    print("\n[7/7] Creating geospatial visualizations...")
    create_geospatial_analysis(processed_df)

    # --- Create Prediction Function ---
    print("\n--- Setting up Prediction Function ---")
    predict_rent_func = create_prediction_function(best_model, best_preprocessor_for_prediction)

    # --- Example Prediction ---
    if predict_rent_func:
        print("\n--- Example Prediction ---")
        # Define sample input - MUST match features used in training!
        # Get column names from the preprocessor's fitted transformers
        try:
             # Get feature names after one-hot encoding etc.
             # This is complex to get perfectly right, especially post-pipeline.
             # Easier: Use the columns from the original X_train dataframe
             example_input_features = X_train.iloc[0].to_dict() # Take the first row of training data as example
             print(f"Using example features: {example_input_features}")

             # Modify a few values for the example prediction
             example_input_features['town'] = 'TAMPINES'
             example_input_features['flat_type'] = '5-ROOM'
             # You might need to add/adjust other features like latitude, longitude, room_count etc.
             # based on the 'TAMPINES'/'5-ROOM' choice, or use averages.
             # For simplicity, we'll use the rest from the original example row.

             predicted_price = predict_rent_func(example_input_features)

             if predicted_price is not None:
                 print(f"\nPredicted rent for example flat ({example_input_features['flat_type']} in {example_input_features['town']}): SGD ${predicted_price:.2f}")
             else:
                 print("\nSample prediction failed.")
        except Exception as e:
             print(f"Error during example prediction: {e}")


    print("\nPipeline execution finished.")

# Ensures that the main() function is called only when the script is executed directly
if __name__ == "__main__":
    main()

Starting Rental Price Prediction Pipeline...

[1/7] Loading data...
Rental data shape: (155464, 6)
Street coordinates data shape: (589, 3)
Income data shape: (25, 3)

[2/7] Exploring data...

Rental Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155464 entries, 0 to 155463
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   rent_approval_date  155464 non-null  object
 1   town                155464 non-null  object
 2   block               155464 non-null  object
 3   street_name         155464 non-null  object
 4   flat_type           155464 non-null  object
 5   monthly_rent        155464 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 7.1+ MB

Rental Data Sample:
  rent_approval_date        town block       street_name flat_type  \
0            2021-01  ANG MO KIO   105  ANG MO KIO AVE 4    4-ROOM   
1            2021-01  ANG MO KIO   107  ANG MO KIO AVE 4    3-ROOM   
2      

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,257 (63.50 KB)

 Trainable params: 16,257 (63.50 KB)

 Non-trainable params: 0 (0.00 B)

Training NN...
Epoch 1/100
3110/3110 - 12s - 4ms/step - loss: 616512.3750 - mae: 539.0908 - val_loss: 277242.2500 - val_mae: 402.5110
Epoch 2/100
3110/3110 - 19s - 6ms/step - loss: 332474.2812 - mae: 444.1286 - val_loss: 272885.7500 - val_mae: 402.0751
Epoch 3/100
3110/3110 - 9s - 3ms/step - loss: 330184.0625 - mae: 442.3075 - val_loss: 283592.3750 - val_mae: 405.5075
Epoch 4/100
3110/3110 - 11s - 3ms/step - loss: 329384.2812 - mae: 441.8941 - val_loss: 275962.0000 - val_mae: 400.6779
Epoch 5/100
3110/3110 - 19s - 6ms/step - loss: 329046.1562 - mae: 441.7492 - val_loss: 273045.0938 - val_mae: 399.5038
Epoch 6/100
3110/3110 - 9s - 3ms/step - loss: 327143.1250 - mae: 439.8705 - val_loss: 273502.7500 - val_mae: 399.4178
Epoch 7/100
3110/3110 - 10s - 3ms/step - loss: 327237.3438 - mae: 439.7630 - val_loss: 270122.5000 - val_mae: 398.6215
Epoch 8/100
3110/3110 - 11s - 4ms/step - loss: 326629.0000 - mae: 439.4319 - val_loss: 271777.6875 - val_mae: 397.1398
Epoch 9/100
3110/3110 - 20s - 7ms/s